In [1]:
from funcs import *

In [ ]:
def eth_circuit(n=2) -> QuantumCircuit:              #1st anc = third last qubit = syndrome/parity measurement qubit, 2nd anc = sec last qubit = magic state for S Gate, 3rd anc = last qubit = magic state for T-Gate
    qr = QuantumRegister(9*n+1, "q")
    cbit = ClassicalRegister(1,"c")
    qc = QuantumCircuit(qr,cbit)

    anc = qc.num_qubits - 1

    qc.h(0)
    qc.h(1)
    qc.h(7)
    qc.h(8)

    # qc.h(4)
    # qc.t(4)
    qc.ry(np.pi/4, 4)
    # qc.x(4)
    for i in range(1):
        w4stab = ClassicalRegister(4)
        qc.add_register(w4stab)
        w2stab = ClassicalRegister(4)
        qc.add_register(w2stab)

        #Z0 Z1 Z3 Z4 Stabilizer:
        qc.reset(anc)
        qc.cx(0, anc)
        qc.cx(1, anc)
        qc.cx(3, anc)
        qc.cx(4, anc)
        qc.measure(anc,w4stab[2])

        #X1 X2 X4 X5 Stabilizer:
        qc.reset(anc)
        qc.h(anc)
        qc.cx(anc, 1)
        qc.cx(anc, 2)
        qc.cx(anc, 4)
        qc.cx(anc, 5)
        qc.h(anc)
        qc.measure(anc,w4stab[0])

        #X3 X4 X6 X7 Stabilizer:
        qc.reset(anc)
        qc.h(anc)
        qc.cx(anc, 3)
        qc.cx(anc, 4)
        qc.cx(anc, 6)
        qc.cx(anc, 7)
        qc.h(anc)
        qc.measure(anc,w4stab[1])

        #Z4 Z5 Z7 Z8 Stabilizer:
        qc.reset(anc)
        qc.cx(4, anc)
        qc.cx(5, anc)
        qc.cx(7, anc)
        qc.cx(8, anc)
        qc.measure(anc,w4stab[3])

        #X0 X1 Stabilizer:
        qc.reset(anc)
        qc.h(anc)
        qc.cx(anc, 0)
        qc.cx(anc, 1)
        qc.h(anc)
        qc.measure(anc,w2stab[0])

        #Z3 Z6 Stabilizer:
        qc.reset(anc)
        qc.cx(3, anc)
        qc.cx(6, anc)
        qc.measure(anc,w2stab[2])

        #X7 X8 Stabilizer:
        qc.reset(anc)
        qc.h(anc)
        qc.cx(anc, 7)
        qc.cx(anc, 8)
        qc.h(anc)
        qc.measure(anc,w2stab[3])

        #Z2 Z5 Stabilizer:
        qc.reset(anc)
        qc.cx(2, anc)
        qc.cx(5, anc)
        qc.measure(anc,w2stab[1])

        # with qc.if_test((w2stab[0],1)):             #0
        #     with qc.if_test((w4stab[0],0)):    
        #         qc.z(0)

        # with qc.if_test((w2stab[0],1)):             #1
        #     with qc.if_test((w4stab[0],1)):
        #         qc.z(1)

        # with qc.if_test((w4stab[0],1)):             #4
        #     with qc.if_test((w4stab[1],1)):
        #         qc.z(4)

        # with qc.if_test((w4stab[1],1)):             #7
        #     with qc.if_test((w2stab[3],1)):
        #         qc.z(7)

        # with qc.if_test((w4stab[1],0)):             #8
        #     with qc.if_test((w2stab[3],1)):
        #         qc.z(8)

        # with qc.if_test((w2stab[0],0)):             #2 und 5
        #     with qc.if_test((w4stab[0],1)):        
        #         with qc.if_test((w4stab[1],0)):    
        #             qc.z(2)

        # with qc.if_test((w4stab[0],0)):             #3 und 6
        #     with qc.if_test((w4stab[1],1)):
        #         with qc.if_test((w2stab[3],0)):
        #             qc.z(3)


        # with qc.if_test((w2stab[2],1)):             #6
        #     with qc.if_test((w4stab[2],0)):
        #         qc.x(6)

        # with qc.if_test((w2stab[2],1)):             #3
        #     with qc.if_test((w4stab[2],1)):
        #         qc.x(3)

        # with qc.if_test((w4stab[2],1)):             #4
        #     with qc.if_test((w4stab[3],1)):
        #         qc.x(4)

        # with qc.if_test((w4stab[3],0)):             #2
        #     with qc.if_test((w2stab[1],1)):
        #         qc.x(2)

        # with qc.if_test((w4stab[3],1)):             #5
        #     with qc.if_test((w2stab[1],1)):
        #         qc.x(5)

        # with qc.if_test((w2stab[2],0)):             #0 und 1
        #     with qc.if_test((w4stab[2],1)):
        #         with qc.if_test((w4stab[3],0)):
        #             qc.x(0)

        # with qc.if_test((w4stab[2],0)):             #7 und 8
        #     with qc.if_test((w4stab[3],1)):
        #         with qc.if_test((w2stab[1],0)):
        #             qc.x(7)
    return qc

def eth_readout(qc: QuantumCircuit, had: int, pos: int, shots: int, noise = 0):
    code0 = ['000110110', '011101101', '110000000', '011011000', '101011000', '000000000', '011101110', '000110101', '101011011', '110110110', '110000011', '101101101', '101101110', '000000011', '011011011', '110110101']
    code1 = ['111001010', '111001001', '100010010', '010010001', '100100111', '001111100', '111111100', '111111111', '001001001', '010100100', '010100111', '010010010', '100010001', '001001010', '100100100', '001111111']

    read = ClassicalRegister(9)
    qc.add_register(read)

    if had%4 == 0:
        for i in range(9):
            qc.measure(i+9*pos, read[8-i])
    elif had%4 == 1:
        qc.measure(0+9*pos, read[8-6])
        qc.measure(1+9*pos, read[8-3])
        qc.measure(2+9*pos, read[8-0])
        qc.measure(3+9*pos, read[8-7])
        qc.measure(4+9*pos, read[8-4])
        qc.measure(5+9*pos, read[8-1])
        qc.measure(6+9*pos, read[8-8])
        qc.measure(7+9*pos, read[8-5])
        qc.measure(8+9*pos, read[8-2])
    elif had%4 == 2:
        for i in range(9):
            qc.measure(i+9*pos, i)
    elif had%4 == 3:
        qc.measure(0+9*pos, read[8-2])
        qc.measure(1+9*pos, read[8-5])
        qc.measure(2+9*pos, read[8-8])
        qc.measure(3+9*pos, read[8-1])
        qc.measure(4+9*pos, read[8-4])
        qc.measure(5+9*pos, read[8-7])
        qc.measure(6+9*pos, read[8-0])
        qc.measure(7+9*pos, read[8-3])
        qc.measure(8+9*pos, read[8-6])

    p = noise
    p_error = pauli_error([["X",p/2],["I",1-p],["Z",p/2]])
    p_error_2 = pauli_error([["XI",p/4],["IX",p/4],["II",1-p],["ZI",p/4],["IZ",p/4]])

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(p_error, ['x', "z", 'h', "id"])  # Apply to single-qubit gates
    noise_model.add_all_qubit_quantum_error(p_error_2, ['cx'])  # Apply to 2-qubit gates

    sim = AerSimulator()
    job = sim.run(qc, noise_model = noise_model, shots=shots)
    result = job.result()
    counts = result.get_counts()

    print(counts)

    register = list(counts.keys())
    register = [i.replace(" ","") for i in register]
    bits = [i[:9] for i in register]
    init1 = [i[-9:-5] for i in register]             #erste Runde QEC: die vier weight-2 stabilizers, müssen alle +1 sein!
    #init2 = [i[-17:-13] for i in register]             #die zweite Runde QEC: vier weight-2 stabilizers, müssen alle +1 sein!
    print("First round QEC, Weight-2 stab: ", init1)
    # print("Second round QEC, Weight-2 stab: ", init2)
    print("bits:", bits)
    print("Anzahl der versch. Bitstring: ", len(bits))
    hmm = list(counts.values())

    for i in range(len(bits)):
        if init1[i].count("1") != 0:
            bits[i] = 2
        # if init2[i].count("1") != 0:
        #     bits[i] = 2
        for j in code0:
            if j == bits[i]:
                bits[i] = 0
                break
        if bits[i] != 0:
            for j in code1:
                if j == bits[i]:
                    bits[i] = 1
                    break
        if bits[i] != 1 and bits[i] != 0:
            bits[i] = 2

    ones = 0
    zeros = 0
    err = 0

    for i in range(len(bits)):
        if bits[i] == 0:
            zeros += hmm[i]
        if bits[i] == 1:
            ones += hmm[i]
        if bits[i] == 2:
            err += hmm[i]
    
    err = (err/shots)
    ones = (ones/shots)/(1-err)
    zeros = (zeros/shots)/(1-err)

    return zeros, ones, err

qc = eth_circuit(n = 1)

anc = qc.num_qubits - 1

#H_L(qc, 0)

zeros, ones, post = eth_readout(qc, had = 0, pos = 0, shots = 10000, noise = 0.00)

#total = zeros + ones

print("Postselection: ", post)
print("With postselec:")
print("Zeros: ", zeros)
print("Ones: ", ones)
# print("Zeros: ", zeros/total)
# print("Ones: ", ones/total)

{'101001000 0000 1101 0': 3, '000100101 0000 1110 0': 9, '011001011 0000 1100 0': 10, '000110100 0000 1000 0': 31, '010100100 0000 0011 0': 5, '111111101 0000 1010 0': 4, '110010000 0000 1111 0': 2, '101101100 0000 1011 0': 31, '011101111 0000 1010 0': 35, '111011010 0000 1101 0': 33, '000000010 0000 1011 0': 32, '100000011 0000 0101 0': 35, '000100110 0000 1110 0': 6, '000010011 0000 1100 0': 3, '100100111 0000 0011 0': 7, '011101101 0000 0011 0': 26, '111011011 0000 0101 0': 33, '010000011 0000 0111 0': 29, '010010001 0000 0010 0': 4, '100110111 0000 1110 0': 33, '100010011 0000 1010 0': 5, '000010010 0000 0111 0': 4, '000110110 0000 0011 0': 34, '111111111 0000 0010 0': 10, '110110110 0000 0001 0': 38, '000000000 0000 0000 0': 25, '100100101 0000 1011 0': 9, '001001000 0000 1010 0': 6, '110000001 0000 1010 0': 35, '100000001 0000 1101 0': 31, '111111110 0000 1010 0': 7, '101101101 0000 0001 0': 36, '010000010 0000 1110 0': 33, '100110101 0000 0100 0': 33, '100110100 0000 1100 0': 35

In [3]:
qr = QuantumRegister(1, "q")
cbit = ClassicalRegister(1,"c")
qc = QuantumCircuit(qr,cbit)

qc.ry(np.pi/4, 0)

qc.measure(0,0)

sim = AerSimulator()
job = sim.run(qc, shots=10000)
result = job.result()
counts = result.get_counts()

print(counts)

{'0': 8551, '1': 1449}


In [31]:
qr = QuantumRegister(9, "q")                #codewords for 0_L and 1_L
cbit = ClassicalRegister(9,"c")
qc = QuantumCircuit(qr,cbit)
for i in range(1):
    qc.h(9*i+1)
    qc.h(9*i+3)
    qc.h(9*i+5)
    qc.h(9*i+7)

    qc.cx(9*i+1,9*i)
    qc.cx(9*i+5,9*i+4)
    qc.cx(9*i+7,9*i+8)

    qc.cx(9*i+5,9*i+2)

    qc.cx(9*i+3,9*i+4)
    qc.cx(9*i+2,9*i+1)

    qc.cx(9*i+3,9*i+6)

    qc.cx(9*i+6,9*i+7)

qc.x(1)
qc.x(4)
qc.x(7)

for i in range(9):
    qc.measure(i,8-i)

sim = AerSimulator()
job = sim.run(qc, shots=100000)
result = job.result()
counts = result.get_counts()

codewords = list(counts.keys())
prob = list(counts.values())

# for i in codewords:
#     if i[1] == 0:
#         i[1] = 1
#     elif i[1] == 1:
#         i[1] = 0
#     if i[4] == 0:
#         i[4] = 1
#     elif i[4] == 1:
#         i[4] = 0
#     if i[7] == 0:
#         i[7] = 1
#     elif i[7] == 1:
#         i[7] = 0
    

print(codewords)
print(prob)

latex = ""
for i in codewords:
    latex += "\ket{"
    latex += i
    latex += "} + "

print(latex)

['010010001', '100010010', '001111100', '001001001', '001001010', '010010010', '100010001', '111001001', '111001010', '010100111', '010100100', '100100100', '001111111', '111111100', '111111111', '100100111']
[6241, 6434, 6279, 6230, 6332, 6276, 6281, 6169, 6237, 6270, 6261, 6197, 6234, 6140, 6199, 6220]
\ket{010010001} + \ket{100010010} + \ket{001111100} + \ket{001001001} + \ket{001001010} + \ket{010010010} + \ket{100010001} + \ket{111001001} + \ket{111001010} + \ket{010100111} + \ket{010100100} + \ket{100100100} + \ket{001111111} + \ket{111111100} + \ket{111111111} + \ket{100100111} + 
